In [17]:
# -*- coding: utf-8 -*-
###
# Demonstration of MR reconstruction with CCP PET-MR Software
#
# This demonstration shows how to reconstruct MR images and how to apply filters 
# in k-space.
#
# This demo is a 'script', i.e. intended to be run step by step in a
# Python IDE such as spyder. It is organised in 'cells'. spyder displays these
# cells nicely and allows you to run each cell on its own.
#
# First version: 27th of May 2017
# Updated: 1st of April 2019
# Author: Christoph Kolbitsch, Johannes Mayer
#

## CCP PETMR Synergistic Image Reconstruction Framework (SIRF).
## Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.
## Copyright 2015 - 2017 University College London.
## Copyright 2015 - 2017 Physikalisch-Technische Bundesanstalt.
##
## This is software developed for the Collaborative Computational
## Project in Positron Emission Tomography and Magnetic Resonance imaging
## (http://www.ccppetmr.ac.uk/).
##
## Licensed under the Apache License, Version 2.0 (the "License");
##   you may not use this file except in compliance with the License.
##   You may obtain a copy of the License at
##       http://www.apache.org/licenses/LICENSE-2.0
##   Unless required by applicable law or agreed to in writing, software
##   distributed under the License is distributed on an "AS IS" BASIS,
##   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
##   See the License for the specific language governing permissions and
##   limitations under the License.

__version__ = '0.1.0'

# import engine module
import pGadgetron as pMR

# import further modules
import os, numpy
%matplotlib
import matplotlib.pyplot as plt



Using matplotlib backend: TkAgg


In [18]:
#%% GO TO MR FOLDER
os.chdir(pMR.petmr_data_path('mr'))



### K-space Filtering
#### Goals of this notebook:
- Definition of our own function in Python.
- Write first lines of code.
- Extraction of the k-space data stored in the AcquisitionData objects.
- Application of a filter to the k-space data.
- Obtain a better understanding of the relationship between k-space and image space.

In [19]:
#%% GO TO MR FOLDER 
os.chdir(pMR.petmr_data_path('mr'))

### Programming task
Now it's time to apply some of our Python knowledge in a minimal programming exercise.
Please write code which completes the following task:
- create a variable named `acq_data` of the class `AcquisitionData` of the `pMR` package.
- the file we want to load is called `simulated_MR_2D_cartesian.h5`. Pass this as the argument to the constructor of `AcquisitionData`.
- create a variable named `kdim` and assign it the return value of the method  `dimensions` of `AcquisitionData`. `dimensions` does not take arguments.

__Please make sure the variable names are as indicated.__

In [20]:
# YOUR CODE GOES HERE





In [14]:
# AS A TEST RUN THIS CELL:
print('Size of k-space %dx%dx%d' % kdim)


NameError: name 'kdim' is not defined

In [21]:
# SOLUTION CELL.
# DON'T RUN UNLESS YOU TRIED TO WRITE YOUR OWN CODE FIRST.
































acq_data = pMR.AcquisitionData('simulated_MR_2D_cartesian.h5')

kdim = acq_data.dimensions()
print('Size of k-space %dx%dx%d' % kdim)




Size of k-space 512x8x512


### Question Data Dimensions:
- Have an quick guess what the different dimensions of the data represent.

In [24]:
#%% PREPROCESS DATA 
# Remember, this will correct for asymmetric echoes, noise correlation and removal of readout oversampling.
preprocessed_data = pMR.preprocess_acquisition_data(acq_data)

# Get size of k-space data after removal of oversampling
kdim = preprocessed_data.dimensions()
print('Size of k-space %dx%dx%d' % kdim)




Size of k-space 512x8x256


### Question Data Dimensions:
- Have an _educated_ guess what the different dimensions of the data represent.

### Image Filter
To filter our image we can use the [convolution theomrem](https://en.wikipedia.org/wiki/Convolution_theorem).  
It states that for two functions $f, g$ and their Fourier transforms $F = \mathcal{F}(f), G = \mathcal{F}(g)$ the following relation holds:  

$$ 
\mathcal{F}(f * g) = F \cdot G
$$

The application of a filter to an image is a convolution of the image with a kernel. This will weight each image pixel with it's neighbouring pixels according to the weight of the kernel.  

Using the convolution theoremt, we can simply weight our k-space data with the Fourier transform of the filter and achieve the same effect.

### A note on Python programming
To define our own function, Python requires the following syntax:
``` 
def function_name( argument1, argument2, ... , argument N):
    something = code_that_does_something_with_arguments
    
    return something
``` 
__Remember that indentation is important in Python!__  

In [26]:
#%% a definition of a Gaussian function
# the numpy package is a powerful algerba package allowing for fast computations
def gaussian(x, mu, sigma):
    return numpy.exp(-numpy.power(x - mu, 2.) / (2 * numpy.power(sigma, 2.)))





In [43]:
#%% CREATE K-SPACE FILTER
# Set width of Gaussian weighting function
sigma = 1.0

# Create 1D Gaussian weighting function 
gauss_1d = gaussian(numpy.array([numpy.linspace(-kdim[2]/2, kdim[2]/2, kdim[2])]),0,sigma)

# Extend 1D weighting function to 2D by repeating it along the phase encoding 
# dimension
gauss_weight = numpy.tile(gauss_1d, (kdim[0], 1))




In [44]:
#%% APPLY K-SPACE FILTER
# Retrieve k-space data
preprocessed_data = pMR.preprocess_acquisition_data(acq_data)
preprocessed_array = preprocessed_data.as_array()

# Apply 2D weighting function to each coil
for c in range(kdim[1]):
    preprocessed_array[:,c,:] = numpy.multiply(preprocessed_array[:,c,:], gauss_weight)
    
# Update modified k-space data    
preprocessed_data.fill(preprocessed_array)




In [45]:
#%% RECONSTRUCT IMAGES
recon = pMR.FullySampledReconstructor()
recon.set_input(preprocessed_data)
recon.process()




In [46]:
#%% RETRIEVE AND VIEW IMAGE
image_data = recon.get_output()
image_array = abs(image_data.as_array())
image_array = image_array/image_array.max()

fig = plt.figure(num=None, figsize=[9, 3])
plt.set_cmap('gray')
ax = fig.add_subplot(1,3,1)
ax.imshow(image_array[0,:,:], vmin=0, vmax=0.7)
ax.set_title('Filtered image, sigma= ' + str(sigma))
ax.axis('off')

ax = fig.add_subplot(1,3,2)
ax.plot(image_array[0,50,:], linewidth=2.0)
ax.set_title('Lineplot')
plt.axis([50, 200, 0, 1])

ax = fig.add_subplot(1,3,3)
ax.plot(gauss_1d[0,:], linewidth=2.0)
ax.set_title('K-space filter')
plt.axis([0, 255, 0, 1])


[0, 255, 0, 1]

### Question:
What are your expectations if you incrase the filter width?  
For a __larger__ width, will the image become:
- more blurry?
- more noisy?


### Programming task
Go back to the cell in which the filter width `sigma` is defined.
- Vary `sigma` and run the next cells again to obtain a differently filtered reconstruction
- view the resulting reconstructions for different kernel widths.


### Question:
What happens for:
- larger filter widths?
- smaller filter widths?

Did this match your expectations?

In [ ]:
plt.close('all')

### Recap
#### We just learned
- to access data stored in AcquisitionData objects.
- which data dimensions correspond to which encoding steps.
- what removal of readout oversampling does.
- to define a Python function.
- to filter an image by weighting its k-space.
- about the reciprocal nature of lenghts in k-space and image space.
